In [1]:
import tweepy
import datetime
import random
import pandas as pd
import numpy as np
import json
import re
from PIL import Image, ImageDraw, ImageFont
import traceback

CK="qwE26UJ39aTUtTsi9JrR444iK"
CS="OGAeBenQukbUC9oHHUCt9cR0Oe66QRYQYkbh0WjKOKluQvzraN"
AT="927590059028578304-vDtXaq3APnFnoXcXLclK4FeTWiIuDmT"
AS="KglCb5ghdGd8MrrGVvY4kx8DAjA2UBwvzluvPKtFM2REp"
 
# Twitterオブジェクトの生成
auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AT, AS)
 
api = tweepy.API(auth)


# バックアップを取る
print "back up..."
now = str(datetime.datetime.now()).replace("-", "").replace(":", "").replace(" ", "").split(".")[0]
bk_name = "shakkin_list_"+now+".csv"
!cp ../shakkin_list/shakkin_list.csv ../shakkin_list/backup/$bk_name
print "success!!"

back up...
success!!


In [3]:
# 関数はここに

# ツイートさせる
def tweet(msg):
    api.update_status(status=msg)
    
# 4文字のハッシュ
def get_random_hash(df):
    s = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    h = ""
    for i in range(4):
        h = h+random.choice(s)
    while h in list(df["id"]):
        h = ""
        for i in range(4):
            h = h+random.choice(s)
    return h


# あいさつ
def greet_mes(text, from_id, user_name):
    mes = ""
    if "おは" in text and len(text) < 30:
        mes = "@"+from_id+" おはようございます、"+user_name+"さん♪"
    if "おやすみ" in text and len(text) < 25:
        mes = "@"+from_id+" おやすみなさい、"+user_name+"さん♪"
    if "帰宅" in text and len(text) < 20:
        mes = "@"+from_id+" おかえりなさい、"+user_name+"さん♪"
    if "ただいま" in text:
        mes = "@"+from_id+" お疲れ様です♪お風呂にしますか？ご飯にしますか？それとも……\nうふふ♡"

    return mes
        
        
with open("../data/Mayu_words.txt", "r") as f:
    lines = f.readlines()
    Mayu_words_db = map(lambda x:x.replace("\n", ""), lines)
        
def Mayu_words(text, from_id, user_name):
    mes = ""
    # あいさつ文があればそれになる
    mes = greet_mes(text, from_id, user_name)
    
    if mes == "":
        if "まゆ" in text:
            if "まゆゆ" in text:
                mes = "@"+from_id+" 違う人ですねぇ…？"
            else:
                mes = "@"+from_id+" まゆですよぉ"
        
        else:
            mes = "@"+from_id+" "+random.choice(Mayu_words_db)
        
    return mes

# エラーメッセージを返す
def error_mes(from_id, tw_id):
    mes = "@"+from_id+" エラーが発生しました…\n考えられる原因\n・文字数オーバー\n・検索対象ツイートが削除された\n・検索対象者にブロックされた など"
    api.update_status(status=mes, in_reply_to_status_id=tw_id)
    return True


# 一覧表示
def get_all_list(from_id, df):
    from_id = "@"+from_id
    
    lend_df = df[df["lender"]== from_id]
    lend_df = lend_df[lend_df["done"]==0.0]

    lend_price = 0
    lend_mes = "  "
    for i in range(len(lend_df)):
        row = lend_df.iloc[i]
        lend_mes =lend_mes + row.price+"円 ( "+ row.content+" ): "+ row.borrower+" へ\n  "
        price = row.price.replace(",", "")
        if price.isdigit():
            lend_price = lend_price + int(price)

    if len(lend_df) == 0:
        lend_mes = "  なし\n"
        
    mes = from_id+ " が貸してる分 (総額: "+str(lend_price)+"円)\n"+lend_mes+"\n"


    borrow_df = df[df["borrower"] == from_id]
    borrow_df = borrow_df[borrow_df["done"] == 0.0]

    borrow_price = 0
    borrow_mes = "  "

    for i in range(len(borrow_df)):
        row = borrow_df.iloc[i]
        borrow_mes =borrow_mes + row.price+"円 ( "+ row.content+" ): "+ row.lender+" から\n  "
        price = row.price.replace(",", "")
        if price.isdigit():
            borrow_price = borrow_price + int(price)

    if len(borrow_df) == 0:
        borrow_mes = "  なし\n"

    mes = mes + from_id + " が借りてる分 (総額: "+str(borrow_price)+"円)\n"+borrow_mes+"\n"
    mes = mes + "※ 総額は計算できるもののみの和を表示"

    return mes
    

# 差額を計算
def calc_diff_price(from_id, man, df):
    
    from_id = "@"+from_id
    man = "@"+man
    
    target_id = ""

    lend_bool = []
    borrow_bool = []
    for i, j, k in zip(list(df["lender"] == from_id), list(df["borrower"] == man), list(df["done"] == 0.0)):
        lend_bool.append(i & j & k)

    for i, j, k in zip(list(df["lender"] == man), list(df["borrower"] == from_id), list(df["done"] == 0.0)):
        borrow_bool.append(i & j & k)

    lend_df = df[lend_bool]
    borrow_df = df[borrow_bool]
    
    unk = "  "
    
    lend_price = 0
    lend_mes = "  "
    for i in range(len(lend_df)):
        row = lend_df.iloc[i]
        price = row.price
        content = row.content
        target_id = target_id+row.id+"/"
        
        if price.replace(",", "").isdigit():
            lend_price = lend_price + int(price.replace(",", ""))
            lend_mes =lend_mes + price+"円 ( "+ content+" )\n  "
        else:
            unk = unk + price+ "円 ( "+content+" ), "+man+" へ\n  "
            
    if lend_price == 0:
        lend_mes = "  なし\n"
        
    mes = man+ " に貸してる分 (総額: "+("{:,}".format(lend_price))+"円)\n"+lend_mes+"\n"

    borrow_price = 0
    borrow_mes = "  "

    for i in range(len(borrow_df)):
        row = borrow_df.iloc[i]
        price = row.price
        content = row.content
        target_id = target_id+row.id+"/"
        
        if price.replace(",", "").isdigit():
            borrow_price = borrow_price + int(price.repace(",", ""))
            borrow_mes =borrow_mes + price+"円 ( "+ content+" )\n  "
        else:
            unk = unk + price+ "円 ( "+content+" ), "+man+" から\n  "

    if borrow_price == 0:
        borrow_mes = "  なし\n"

    mes = mes + man+" から借りてる分 (総額: "+("{:,}".format(borrow_price))+"円)\n"+borrow_mes+"\n"
    mes = mes + "※ 総額、差額は計算できるもののみを利用"

    if len(unk) > 2:
        mes = mes + "\n\n金額不明:\n"+unk
    
    diff_price = lend_price-borrow_price
    if diff_price == 0:
        mes = "差額はゼロです♡\n\n"+mes
    if diff_price > 0:
        mes = "差額は　"+str(diff_price)+"円です♡\n"+from_id+" が "+man+" から返してもらってくださいね。\n\n"+mes
    if diff_price < 0:
        mes = "差額は "+str(-diff_price)+"円です♡\n"+from_id+" が "+man+" へ返してくださいね。\n\n"+mes

    return mes, target_id


# ツイートするための画像を生成
def make_image(mes, use):
    base_image = Image.open("../images/base.png")
    draw = ImageDraw.Draw(base_image)
    f_size = 20
    b = (0, 0, 0)
    font = ImageFont.truetype("/Library/Fonts/ipaexg.ttf", f_size, encoding='utf-8')
    
    draw.text((10, 10), mes.decode("utf-8"), b, font=font)
    
    now = str(datetime.datetime.now()).replace("-", "").replace(":", "").replace(" ", "").split(".")[0]
    
    if use == "all_list":
        filepath = "../images/all_list/"+now+".png"
    if use == "diff_price":
        filepath = "../images/diff_price/"+now+".png"

    base_image.save(filepath)
    
    return filepath

def omikuji():
    mes = ""
    
    daikichi = 0.01
    kichi = 0.03
    kyou = 1.0 - daikichi - kichi
    rnd = random.random()
    
    if rnd < daikichi:
        mes = "大吉です♡"
    elif rnd < kichi:
        mes = "吉です♪"
    else:
        mes = "凶です…"
    return mes

# TL の特定ワードに反応するやつ
# 優先順位が高いやつほど、下に追加
def word_reaction_TL(text, from_id, user_name):
    mes = ""

    flag = True
    if "まゆ" in text:
        mes = "@"+from_id+" "+"まゆですよぉ"
        flag = False
    if "まゆゆ" in text:
        mes = "@"+from_id+" "+"違う人ですねぇ…？"
        flag = False
    
    rnd = random.random()
    if rnd < 0.7 and flag:
        mes = greet_mes(text, from_id, user_name)
    
    return mes

# ららマジの曜日プレゼント
def rara_magi():
    # 曜日
    today = datetime.datetime.now().weekday()
    
    weekday = ["月曜日", "火曜日", "水曜日", "木曜日", "金曜日", "土曜日", "日曜日"]
    kigakubu = ["亜里砂・E・B\n卯月真中華\n奥村映\n白石陽菜\n洲崎麻衣\n七瀬沙希\n結城菜々美",
               "阿達悠花\n綾瀬凜\n楓智美\n神田茜\n島村珠樹\n洲崎麻衣\n園田乃愛\n藤巻雪菜",
               "浅野葉月\n阿達悠花\n亜里砂\nE\nB\n有栖川翼\n卯月幸\n神代結菜\n神田茜\n九条紗彩\n白石陽菜\n橘レイナ",
               "綾瀬凜\n伊藤萌\n奥村映\n卯月真中華\n楓智美\n橘アンナ\n七瀬沙希\n南さくら",
               "浅野葉月\n伊藤萌\n瀬沢かなえ\n橋本ひかり\n藤巻雪菜\n星崎梨花\n向井春香",
                "小田桐アミ\n神代結菜\n瀬沢かなえ\n橘アンナ\n橘レイナ\n月島塁\n星崎梨花\n三嶋蒼\n南さくら\n向井春香",
                "有栖川翼\n卯月幸\n小田桐アミ\n九条紗彩\n島村珠樹\n園田乃愛\n月島塁\n橋本ひかり\n三嶋蒼\n結城菜々美"
               ]
    present = ["ブランド肉の贅沢ステーキ", "一昔前に流行した携帯型ゲーム", "おもちゃの指輪", "週刊お兄ちゃん",
                      "意識高い系ファッション誌", "月刊フー", "月刊フー"]
    
    mes = weekday[today]+": "+present[today]+"\n"+kigakubu[today]
    return mes


# ワード検索
def search_word(w, from_id, df):
    from_id = "@"+from_id
    search_df = df[map(lambda x: (x[0] | x[1]) & x[2] & (w in x[3]) , zip(list(df.lender==from_id), list(df.borrower==from_id), list(df.done==0.0), list(df.content)))]
    
    mes = ""
    for i in range(len(search_df)):
        tmp = search_df.iloc[i]
        mes = mes + "https://twitter.com/"+tmp.from_id.replace("@", "")+"/status/"+str(tmp.tweet_id)+"\n"
    
    return mes[:-1]
    
# リプライの特定ワードに反応するやつ
# 引数: 反応させたい特定ワードとtext
def word_reaction_reply(text, tw_id, from_id, member, df):
    
    # 検索
    if "検索" in text:
        w = text.split("検索")[1]
        search_list = search_word(w, from_id, df)
        mes = "@"+from_id+" これですかぁ？\n"+search_list
        if search_list == "":
            mes = "@"+from_id+" 指定の検索ワードでの登録は見つかりませんでした…"
            
        try:
            api.update_status(status=mes, in_reply_to_status_id=tw_id)
        except:
            error_mes(from_id, tw_id)
            
        return True
    
    # 一覧を出す
    if "一覧" in text:
        tmp = get_all_list(from_id, df)
        filepath = make_image(tmp, "all_list")
        mes = "@"+from_id+" どうぞ♡"
        api.update_with_media(status=mes, in_reply_to_status_id=tw_id, filename=filepath)
        return True

    # 差額を出す
    if "差額" in text and len(member) > 0:
        for man in member:
            tmp, tmp2 = calc_diff_price(from_id, man, df)
            if tmp2 == "":
                tmp2 = "なし"
            filepath = make_image(tmp, "diff_price")
            # print filepath
            mes = "@"+from_id+" .@"+man+" さんとの差額\nID: "+tmp2
            try:
                api.update_with_media(status=mes, in_reply_to_status_id=tw_id, filename=filepath)
            except:
                error_mes(from_id, tw_id)
                
        return True

    # おみくじ10連
    if  "10連" in text:
        mes = "@"+from_id+" "
        for i in range(10):
            mes = mes+omikuji()+"\n"
        
        if "大吉" in mes:
            mayu_pictures =  !ls /Users/Shota/Documents/jupyter/otaku_shakkin_chang/data/Mayu_pic/
            filepath = random.choice(mayu_pictures)
            api.update_with_media(status=mes, in_reply_to_status_id=tw_id, filename=filepath)
        else:
            api.update_status(status=mes, in_reply_to_status_id=tw_id)
            
        return True

    # おみくじ 
    if "おみくじ" in text:
        mes = "@"+from_id+" "+omikuji()
        api.update_status(status=mes, in_reply_to_status_id=tw_id)
        return True
    
    if "大吉" in text:
        mes = "@"+from_id+" 大吉です♪"
        mayu_pictures =  !ls /Users/Shota/Documents/jupyter/otaku_shakkin_chang/data/Mayu_pic/
        filepath = random.choice(mayu_pictures)
        api.update_with_media(status=mes, in_reply_to_status_id=tw_id, filename=filepath)
        return True
        
    # ららマジ
    if "ららマジ" in text:
        mes = "@"+from_id+" "+rara_magi()
        api.update_status(status=mes, in_reply_to_status_id=tw_id)
        return True
        
    # どれにも一致しなかったら false 
    return False

In [4]:
path = "../shakkin_list/shakkin_list.csv"
bot_id = "otaku_shakkin"

class Listener(tweepy.StreamListener):

    # status がツイートに関する情報
    def on_status(self, status):
        # 借金リストの読み込み
        df = pd.read_csv(path, encoding="utf-8", parse_dates=[0])
        columns = df.columns
        
        # ツイート ID
        tw_id = status.id
        
        # 日時
        tw_date = status.created_at + datetime.timedelta(hours=9)
        
        # 誰からのリプライか
        from_id = status.author.screen_name
        # ユーザネーム
        user_name = status.author.name
        
        # オタク借金ちゃんへのリプライか？
        # デフォルトは False (=TL に流れてるだけ)
        is_reply = False
        
        # オタク借金ちゃん自身の投稿には反応しない
        if from_id == bot_id:
            # print 1
            return True
        
        # 誰へのリプライか？
        # オタク借金ちゃんへのリプライか？
        member = []
        for i in status._json["entities"]["user_mentions"]:
            user_id = i["screen_name"]
            if user_id == bot_id:
                is_reply = True
            else:
                member.append(user_id)
                
        # print is_reply
        # print member
        # print "len(member) =", len(member)
                
        # @ID を全部抜いたテキストを抽出、空白も除去
        text = status.text.replace("@"+bot_id, "")
        for i in member:
            text = text.replace("@"+i, "")
        text = re.sub(r'\s', "", text)
        
        # print text
        
        # 以下は、オタク借金ちゃんを含むリプライの場合のみ
        
        if is_reply:

            # リプライの特定ワードに反応するやつ
            if word_reaction_reply(text, tw_id, from_id, member, df):
                return True
            
            # 以下は特殊                
            # 借金の更新
            if "済" in text:
                # いきなり 済み って送られてきたパターンじゃないとき
                if status.in_reply_to_status_id != None:
                    # ここでデータフレームの done を書き換えるやつ
                    # リプライ元のツイート取得
                    tw_in_rep = api.get_status(id=status.in_reply_to_status_id)
                    
                    # 全部済みのやつ、差額計算ツイートに対してのみ有効
                    if "全部済" in text:
                        # そのリプライ元がオタク借金ちゃんのツイートで、かつ差額を計算したときのやつか？
                        if tw_in_rep.author.screen_name == bot_id and "さんとの差額\nID: " in tw_in_rep.text:
                            
                            # 過去の差額ツイにリプライされると困る
                            try:
                                done_ids = tw_in_rep.text.split("ID: ")[-1].split(" ")[0].split("/")[:-1]
                            except:
                                mes = "@"+from_id+" 何か問題が起きたようです…\n差額計算からの更新をしたい場合は、もう一度新しく差額を聞いてみてからにしてください。"
                                api.update_status(status=mes, in_reply_to_status_id=tw_id)
                                return True

                            # ひとつでも更新したか？
                            update_num = 0
                            for done_id in done_ids:
                                if len(done_id) > 0:
                                    
                                    # バックアップ
                                    now = str(datetime.datetime.now()).replace("-", "").replace(":", "").replace(" ", "").split(".")[0]
                                    bk_name = "shakkin_list_"+now+".csv"
                                    !cp ../shakkin_list/shakkin_list.csv ../shakkin_list/backup/$bk_name
                                    
                                    update_row = df[df.id==done_id]
                                    reply_id = list(update_row.reply_id)[0]
                                    
                                    # 借金に関わってる人か？
                                    if list(update_row["borrower"] == "@"+from_id)[0] or list(update_row["lender"] == "@"+from_id)[0]:
                                        df.loc[df.id == done_id, "done"] = 1
                                        try:
                                            api.update_status(status="これは済みです♡\n"+str(datetime.datetime.now()), in_reply_to_status_id=reply_id)
                                            update_num = update_num + 1
                                        except:
                                            pass
                                    else:
                                        mes = "@"+from_id+" "+"あなたはこの借金に関わってませんねぇ…？"
                                        api.update_status(status=mes, in_reply_to_status_id=tw_id)
                                        return True
                            
                            if update_num > 0:
                                df.to_csv(path, encoding="utf-8", index=False)
                                mes = "@"+from_id+" 全て更新完了しました♡"
                                api.update_status(status=mes, in_reply_to_status_id=tw_id)
                                return True
                            else:
                                mes ="@"+from_id+" 更新できるものはありませんでした…"
                                api.update_status(status=mes, in_reply_to_status_id=tw_id)
                                return True
                        else:
                            mes = "@"+from_id+" 差額をお知らせしたものに対してツイートしてくださいね♡"
                            api.update_status(status=mes, in_reply_to_status_id=tw_id)
                            return True
                            
                                    
                    # 全部済 ではない普通の済み
                    # そのリプライ元がオタク借金ちゃんのツイートで、かつちゃんと登録したときのツイートか？
                    if tw_in_rep.author.screen_name == bot_id and "ID: " in tw_in_rep.text:
                        # バックアップ
                        now = str(datetime.datetime.now()).replace("-", "").replace(":", "").replace(" ", "").split(".")[0]
                        bk_name = "shakkin_list_"+now+".csv"
                        !cp ../shakkin_list/shakkin_list.csv ../shakkin_list/backup/$bk_name
                        
                        done_id = tw_in_rep.text.split("ID: ")[-1]
                        update_row = df[df.id == done_id]
                        
                        # その借金の貸し借りに関わっている人しか完了できない
                        if list(update_row["borrower"] == "@"+from_id)[0] or list(update_row["lender"] == "@"+from_id)[0]:
                            df.loc[df.id == done_id, "done"] = 1
                            df.to_csv(path, encoding="utf-8", index=False)
                            # print 4
                            mes = "@"+from_id+" "
                            for i in member:
                                # print member
                                mes = mes + "@"+ i +" "
                            mes = mes + "\n更新完了です！"
                            api.update_status(status=mes, in_reply_to_status_id=tw_id)
                            return True
                        else:
                            mes = "@"+from_id+" "+"あなたはこの借金に関わってませんねぇ…？"
                            api.update_status(status=mes, in_reply_to_status_id=tw_id)
                            return True
                        
            
            if "円" in text:
                if len(member) > 0:
                    for i in member:
                        if i != bot_id:
                            # ここで df への登録
                            # print 7
                            lender_id = from_id
                            borrower_id = i
                            if "逆" in text:
                                if len(member) == 1:
                                    tmp = lender_id
                                    lender_id = borrower_id
                                    borrower_id = lender_id
                                    text = text.replace("逆", "")
                                else:
                                    mes = "@"+from_id+" 意図した登録にならないかもしれません…\n逆登録の場合は一人に対してしかできません。"
                                    api.update_status(status=mes, in_reply_to_status_id=tw_id)
                                    return True

                            # バックアップ
                            now = str(datetime.datetime.now()).replace("-", "").replace(":", "").replace(" ", "").split(".")[0]
                            bk_name = "shakkin_list_"+now+".csv"
                            !cp ../shakkin_list/shakkin_list.csv ../shakkin_list/backup/$bk_name
                            
                            price = text.split("円")[0]
                            if price.isdigit():
                                price = "{:,}".format(int(price))
                            content = text.split("円")[-1]
                            rh = get_random_hash(df)
                            mes = "登録完了です！\n@"+lender_id+" から "+"@"+borrower_id+" へ\n"+price+"円: "+content+"\n#オタク借金\nID: "+rh
                            api.update_status(status=mes, in_reply_to_status_id=tw_id)
                            
                            # 返信ツイートの ID を取得
                            reply_id = api.user_timeline()[0].id
                            # columns = [u'date', u'tweet_id', u'from_id', u'reply_id', u'id', u'price', u'content', u'lender', u'borrower', u'done']
                            row = pd.DataFrame([[tw_date, str(tw_id), "@"+from_id, str(reply_id), rh, price, content, "@"+lender_id, "@"+borrower_id, 0]], columns=columns)
                            df = df.append(row, ignore_index=True)

                    df.to_csv(path, encoding="utf-8", index=False)
                    return True
                
                
            # ただのリプライのとき    
            # print 9
            mes = Mayu_words(text, from_id, user_name)
            api.update_status(status=mes, in_reply_to_status_id=tw_id)
            return True
            
        
        # TL に流れてるやつ
        mes = word_reaction_TL(text, from_id, user_name)
        if mes != "" and len(member) == 0:
            # print 8
            api.update_status(status=mes, in_reply_to_status_id=tw_id)
            return True
        
        # 最後
        return True
        
    def on_error(self, status_code):
        print 'Got an error with status code: ' + str(status_code)
        return True
     
    def on_timeout(self):
        print 'Timeout...'

In [ ]:
# 本番用
print "start at", datetime.datetime.now()
while True:
    listener = Listener()
    stream = tweepy.Stream(auth, listener)
    stream.timeout = 300.0
    try:
        stream.userstream()
    except:
        traceback.print_exc()
        print "再起動", datetime.datetime.now()

start at 2018-01-31 01:27:08.351479


In [4]:
# デバッグ用
listener = Listener()
stream = tweepy.Stream(auth, listener)
stream.timeout = 300.0
stream.userstream()

greet mes


KeyboardInterrupt: 